In [1]:
import os
os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

In [2]:
from langchain.document_loaders import DirectoryLoader

loader = DirectoryLoader('./my-app/', glob="**/*.md")

In [14]:
docs = loader.load()

In [15]:
docs = list(filter(lambda x: 'node_modules' not in x.metadata['source'], docs))

In [21]:
from langchain.chains import SequentialChain, LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.7)

template = """
Given the following descriptions of what directories are for and their file conventions,
output all the files needed to add {feature_description} to the app. For each file, include
the description of what the file is for.


{directory_description}

The reponse format should be a list of file descriptions, like this:
1. path/to/file1.js: description of file1
...
"""
which_files_prompt = PromptTemplate(
    input_variables=["directory_description", "feature_description"],
    template=template,
)

chain = LLMChain(prompt=which_files_prompt, llm=llm, verbose=True)

predicted_files = chain.predict(feature_description="create a new view for a user's posts", directory_description=docs)



> Entering new LLMChain chain...
Prompt after formatting:

Given the following descriptions of what directories are for and their file conventions,
output all the files needed to add create a new view for a user's posts to the app. For each file, include
the description of what the file is for.


[Document(page_content='This is a Next.js project bootstrapped with create-next-app.\n\nGetting Started\n\nFirst, run the development server:\n\n```bash\nnpm run dev\n\nor\n\nyarn dev\n\nor\n\npnpm dev\n```\n\nOpen http://localhost:3000 with your browser to see the result.\n\nYou can start editing the page by modifying app/page.js. The page auto-updates as you edit the file.\n\nAPI routes can be accessed on http://localhost:3000/api/hello. This endpoint can be edited in pages/api/hello.js.\n\nThe pages/api directory is mapped to /api/*. Files in this directory are treated as API routes instead of React pages.\n\nThis project uses next/font to automatically optimize and load Inter, a custom G

In [26]:
for file in predicted_files.split('\n'): 
    print(file)


1. app/pages/posts/[id].tsx: React component to create a view for a user's post
2. app/pages/posts/[id].scss: styles associated with the posts view
3. app/pages/posts/[id].test.ts: tests for the posts view


In [28]:
files = predicted_files.split('\n')
files[1]

"1. app/pages/posts/[id].tsx: React component to create a view for a user's post"